In [2]:
import numpy as np
import glob
import os
import torch
import faiss
import os, pandas as pd, time, random
import tiktoken
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from elasticsearch import Elasticsearch


#  読み込みと設定

In [3]:
vector_ids = np.load("/disks/qnap3/shared/openalex-24/data/KNN/all_ids.npy")
index = faiss.read_index("/disks/qnap3/shared/openalex-24/data/KNN/all.faiss")
index.nprobe = 5

def search_and_get_ids(query_vector, k=300):
    query_vector = np.asarray(query_vector, dtype='float32')
    if query_vector.ndim == 1:
        query_vector = query_vector[None, :]   # または query_vector.reshape(1, -1)
    D, I = index.search(query_vector, k)
    result_ids = [vector_ids[idx] for idx in I[0] if idx != -1]
    res = list(D[0]), list(result_ids)
    res = pd.DataFrame(res).T
    res.columns = ['score','id']
    res['id'] = res['id'].apply(int)
    return res

es = Elasticsearch(
    "http://133.11.72.200:9200/",
    # ca_certs="./ti06.crt",
    timeout=500,
    basic_auth=('oa_reader', 'five@godtruth')
)

/tmp/ipykernel_143/3614551544.py:17: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(


#  任意の論文IDから近い論文IDを検索

In [4]:
# setting

es = Elasticsearch(
    "http://133.11.72.200:9200/",
    # ca_certs="./ti06.crt",
    timeout=500,
    basic_auth=('oa_reader', 'five@godtruth')
)

def get_documents_by_ids(es, index_name, doc_ids):
    # Use the mget API to retrieve multiple documents by IDs
    response = es.mget(
        index=index_name,
        body={"ids": doc_ids}
    )
    
    # Process and return the results
    results = []
    for doc in response['docs']:
        if doc.get('found', False):
            results.append(doc['_source'])
        else:
            print(f"Document with ID {doc['_id']} not found")
    
    return pd.DataFrame(results)
    

# get text embedding of the paper
paper_id = 3134007932
paper_data = get_documents_by_ids(es,'oa_papers',[paper_id])
emb = paper_data.iloc[0].text_embedding_256

# retrieve the KNN
res = search_and_get_ids(emb, 300)
res

/tmp/ipykernel_143/357058807.py:3: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(


,score,id
0,0.986877,3134007932
1,0.880838,4248341347
2,0.831558,4398217026
3,0.822026,4386961494
4,0.819856,4391448611
...,...,...
295,0.730429,3013793046
296,0.729953,4390137167
297,0.729904,2530840575
298,0.729584,2900977896


In [5]:
# you can access the paper data using following url
# https://openalex.org/works/W{id}
# e.g. https://openalex.org/works/W{3134007932}

#  任意のテキストから近い論文IDを検索


In [7]:
# setting 
os.environ["OPENAI_API_KEY"] = "YOURKEY"
client = OpenAI()
enc = tiktoken.encoding_for_model("text-embedding-3-small")

def get_embeddings(client, enc, texts, model="text-embedding-3-small", dimensions=None):
    df = pd.Series(texts)
    df_ = df.dropna()
    df_ = df_[df_.apply(len) > 0]
    df = df_[df_.apply(enc.encode).apply(len) < 8000]
    texts = list(df_)
    texts = [text.replace("\n", " ") for text in texts]
    try:
        if dimensions:
            res = client.embeddings.create(input=texts, model=model, dimensions=dimensions)
        else:
            res = client.embeddings.create(input=texts, model=model)
        emb = [x.embedding for x in res.data]
        df = pd.DataFrame(np.stack(emb))
        df.index = df_.index
        return df
    except Exception as e:
        print()
    return pd.Series()

# your text
text = """quantum computing"""
# get embedding using openAI api. dimensions should be 256 
embs = get_embeddings(client,enc,[text],dimensions=256).iloc[0]

# KNN search
res = search_and_get_ids(embs, 300)
res

IndexError: single positional indexer is out-of-bounds